In [ ]:
import tensorflow as tf

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    print("TPU подключен.")
except ValueError:
    print("TPU не найден. Проверь настройки среды выполнения.")


TPU подключен.


In [ ]:
from google.colab import files
files.upload()  # Загрузить kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d harlequeen/synthetic-rgbd-images-of-plants
!unzip synthetic-rgbd-images-of-plants.zip

In [ ]:
!pip install OpenEXR numpy pillow
import OpenEXR
import Imath
import numpy as np
from PIL import Image
import cv2
import imageio.v2 as imageio
from PIL import Image
import numpy as np
from keras.preprocessing.image import img_to_array
import tensorflow.keras.backend as K

from keras.layers import Layer
from keras.layers import Input
from keras.layers import Convolution2D
from keras.layers import Activation, Reshape
from keras.layers import BatchNormalization
from keras.layers import add
from keras.layers import Dropout
from keras.models import Model
from keras.layers import Concatenate
from keras.layers import Conv2DTranspose
from keras.layers import UpSampling2D
from keras.layers import Average
import keras.optimizers
import argparse
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.layers import MaxPooling2D, UpSampling2D


In [ ]:
def unetpp(input_shape, n_labels, kernel=3,pool_size=(2, 2),output_mode="softmax"):
  inputs = Input(shape=input_shape)

  X00 = MaxPooling2D(pool_size)(inputs)
  X00 = Convolution2D(32,(kernel,kernel),padding = "same")(X00)
  X00 = BatchNormalization()(X00)
  X00 = Activation("relu")(X00)

  X10 = MaxPooling2D(pool_size)(X00)
  X10 = Convolution2D(64,(kernel,kernel),padding = "same")(X10)
  X10 = BatchNormalization()(X10)
  X10 = Activation("relu")(X10)

  tmpX10 = UpSampling2D(pool_size)(X10)


  X01 = Convolution2D(32,(kernel,kernel),padding = "same")(Concatenate()([X00,tmpX10]))
  X01 = BatchNormalization()(X01)
  X01 = Activation("relu")(X01)

  X20 = MaxPooling2D(pool_size)(X10)
  X20 = Convolution2D(128,(kernel,kernel),padding = "same")(X20)
  X20 = BatchNormalization()(X20)
  X20 = Activation("relu")(X20)

  tmpX20 = UpSampling2D(pool_size)(X20)
  X11 = Convolution2D(64,(kernel,kernel),padding = "same")(Concatenate()([X10,tmpX20]))
  X11 = BatchNormalization()(X11)
  X11 = Activation("relu")(X11)


  tmpX11 = UpSampling2D(pool_size)(X11)
  X02 = Convolution2D(32,(kernel,kernel),padding = "same")(Concatenate()([X00, X01,tmpX11]))
  X02 = BatchNormalization()(X02)
  X02 = Activation("relu")(X02)

  X30 = MaxPooling2D(pool_size)(X20)
  X30 = Convolution2D(256,(kernel,kernel),padding = "same")(X30)
  X30 = BatchNormalization()(X30)
  X30 = Activation("relu")(X30)

  tmpX30 = UpSampling2D(pool_size)(X30)
  X21 = Convolution2D(128,(kernel,kernel),padding = "same")(Concatenate()([X20,tmpX30]))
  X21 = BatchNormalization()(X21)
  X21 = Activation("relu")(X21)


  tmpX21 = UpSampling2D(pool_size)(X21)
  X12 = Convolution2D(64,(kernel,kernel),padding = "same")(Concatenate()([X10,X11,tmpX21]))
  X12 = BatchNormalization()(X12)
  X12 = Activation("relu")(X12)


  tmpX12 = UpSampling2D(pool_size)(X12)
  X03 = Convolution2D(32,(kernel,kernel),padding = "same")(Concatenate()([X00,X01,X02,tmpX12]))
  X03 = BatchNormalization()(X03)
  X03 = Activation("relu")(X03)


  X40 = MaxPooling2D(pool_size)(X30)
  X40 = Convolution2D(512,(kernel,kernel),padding = "same")(X40)
  X40 = BatchNormalization()(X40)
  X40 = Activation("relu")(X40)

  tmpX40 = UpSampling2D(pool_size)(X40)
  X31 = Convolution2D(128,(kernel,kernel),padding = "same")(Concatenate()([X30,tmpX40]))
  X31 = BatchNormalization()(X31)
  X31 = Activation("relu")(X31)


  tmpX31 = UpSampling2D(pool_size)(X31)
  X22 = Convolution2D(128,(kernel,kernel),padding = "same")(Concatenate()([X20,X21,tmpX31]))
  X22 = BatchNormalization()(X22)
  X22 = Activation("relu")(X22)


  tmpX22 = UpSampling2D(pool_size)(X22)
  X13 = Convolution2D(64,(kernel,kernel),padding = "same")(Concatenate()([X10,X11,X12,tmpX22]))
  X13 = BatchNormalization()(X13)
  X13 = Activation("relu")(X13)

  tmpX13 = UpSampling2D(pool_size)(X13)
  X04 = Convolution2D(32,(kernel,kernel),padding = "same")(Concatenate()([X00,X01,X02,X03,tmpX13]))
  X04 = BatchNormalization()(X04)
  X04 = Activation("relu")(X04)
  X04 = Conv2DTranspose(5,(kernel,kernel),strides = (2,2),padding = "same")(X04)
  outputs = Activation(output_mode)(X04)
  print("Build decoder done..")

  model = Model(inputs=inputs, outputs=outputs, name="UNetPlusPlus",output_mode="softmax")

  return model

with strategy.scope():
  model = unetpp((224,224,3), n_labels=5 ,kernel=3, pool_size=(2,2))
print(model.summary())


Build decoder done..


Model: "UNetPlusPlus"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_20          │ (None, 112, 112, 3)    │              0 │ input_layer_4[0][0]    │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_60 (Conv2D)        │ (None, 112, 112, 32)   │            896 │ max_pooling2d_20[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_60    │ (None, 112, 112, 32)   │            128 │ conv2d_60[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_61             │ (None, 112, 112, 32)   │              0 │ batch_normalization_6… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_21          │ (None, 56, 56, 32)     │              0 │ activation_61[0][0]    │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_61 (Conv2D)        │ (None, 56, 56, 64)     │         18,496 │ max_pooling2d_21[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_61    │ (None, 56, 56, 64)     │            256 │ conv2d_61[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_62             │ (None, 56, 56, 64)     │              0 │ batch_normalization_6… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_22          │ (None, 28, 28, 64)     │              0 │ activation_62[0][0]    │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_63 (Conv2D)        │ (None, 28, 28, 128)    │         73,856 │ max_pooling2d_22[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_63    │ (None, 28, 28, 128)    │            512 │ conv2d_63[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_64             │ (None, 28, 28, 128)    │              0 │ batch_normalization_6… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_23          │ (None, 14, 14, 128)    │              0 │ activation_64[0][0]    │
│ (MaxPooling2D)            │                        │                │                        │
├──────────────────────

 Total params: 3,955,237 (15.09 MB)

 Trainable params: 3,951,845 (15.08 MB)

 Non-trainable params: 3,392 (13.25 KB)

None


In [ ]:
def exr_to_jpg(path):
    exr_file = OpenEXR.InputFile(path)

    header = exr_file.header()
    data_window = header['dataWindow']
    width = data_window.max.x - data_window.min.x + 1
    height = data_window.max.y - data_window.min.y + 1


    channels = ["R","G","B"]
    data = [np.frombuffer(exr_file.channel(channel, Imath.PixelType(Imath.PixelType.FLOAT)), dtype=np.float32)
                for channel in channels]
    print(data,data[0].mean(), data[0].var(), data[0].min(),data[0].max())
    array = [np.clip(channel.reshape(height, width)/(np.max(channel)+ 1e-7),0,1).astype(np.float32) for channel in data]

    image = np.dstack(array)
    return image

def category_label(labels, dims, n_labels):
    x = np.zeros([dims[0], dims[1], n_labels])
    for i in range(dims[0]):
        for j in range(dims[1]):
            f=int(labels[i,j])
            x[i, j, f] = 1
    return x
def colorize(img):
    w=img.shape[0]
    h=img.shape[1]
    z=img.shape[2]
    l=np.zeros((w,h,3))
    for i in range(w):
        for j in range(h):
            if img[i,j,0]==1:
                l[i,j,0]=0
                l[i,j,1]=0
                l[i,j,2]=0
            elif img[i,j,1]==1:
                l[i,j,0]=255
                l[i,j,1]=0
                l[i,j,2]=0
            elif img[i,j,2]==1:
                l[i,j,0]=0
                l[i,j,1]=255
                l[i,j,2]=0
            elif img[i,j,3]==1:
                l[i,j,0]=0
                l[i,j,1]=0
                l[i,j,2]=255
            elif img[i,j,4]==1:
                l[i,j,0]=238
                l[i,j,1]=197
                l[i,j,2]=145
    return l

def class_pixels(img):
    w=img.shape[0]
    h=img.shape[1]
    z=img.shape[2]
    l=np.zeros((w,h,z))
    for i in range(w):
        for j in range(h):
            for f in range(z-1):
                if img[i,j,f]==np.max([img[i,j,0],img[i,j,1],img[i,j,2],img[i,j,3],img[i,j,4]]):
                    l[i,j,f]=1
    return l


In [ ]:
import concurrent.futures
def read_exr(file_path):
    exr_file = OpenEXR.InputFile(file_path)
    header = exr_file.header()
    dw = header['dataWindow']
    size = (dw.max.x - dw.min.x + 1, dw.max.y - dw.min.y + 1)

    # Читаем все каналы (RGB)
    channels = exr_file.channels("RGB", Imath.PixelType(Imath.PixelType.FLOAT))
    r = np.frombuffer(channels[0], dtype=np.float32).reshape(size[1], size[0])
    g = np.frombuffer(channels[1], dtype=np.float32).reshape(size[1], size[0])
    b = np.frombuffer(channels[2], dtype=np.float32).reshape(size[1], size[0])

    # Объединяем каналы в изображение
    img = np.stack([r, g, b], axis=-1)
    return img
def process_file(index, img_dir, mask_dir, dims, n_labels):
    img_path = img_dir[index]
    original_img = exr_to_jpg(img_path) #read_exr(img_path) #exr_to_jpg(img_path)
    array_img = img_to_array(original_img)
    #array_img = (array_img - mean) / (std)

    mask_path = mask_dir[index]
    original_mask = read_exr(mask_path)
    array_mask = category_label(original_mask[:, :, 0], dims, n_labels)

    return array_img, array_mask

def data(img_dir, mask_dir, depth_dir, liste, dims=(224, 224), n_labels=5):
  imgs,lables = [],[]
  for i in liste:
    img,mask = process_file(i,img_dir, mask_dir, dims, n_labels)
    imgs.append(img)
    lables.append(mask)
  imgs = tf.convert_to_tensor(imgs,dtype=tf.float32)
  lables = tf.convert_to_tensor(lables,dtype=tf.float32)
  return imgs,lables

In [ ]:
import os
with strategy.scope():
  rgb=[]
  depth=[]
  mask=[]
  node=[]
  for dirs,subdir,files in os.walk('/content/dataset of synthetic rgb-d plants/rgb_map'):
      for file_name in files:
        if file_name.endswith(".exr"):
              path_file=dirs+os.sep+file_name
              depth_file='/content/dataset of synthetic rgb-d plants/depth_map/profondeur_map/'+file_name
              mask_file='/content/dataset of synthetic rgb-d plants/semantic_map/segmentation2_map/'+file_name
              node_file='/content/dataset of synthetic rgb-d plants/nodes_map/internoeuds_map/'+file_name
              rgb.append(path_file)
              depth.append(depth_file)
              mask.append(mask_file)
              node.append(node_file)




  liste=np.arange(1,10000)
  np.random.shuffle(liste)

  train_list=liste[0:6000]
  val_list=liste[6000:9000]
  test_list=liste[9000:9600]


  train_data = tf.data.Dataset.from_tensor_slices(data(rgb,mask,depth,liste=train_list,dims=(224,224),n_labels=5))
  val_data = tf.data.Dataset.from_tensor_slices(data(rgb,mask,depth,liste=val_list,dims=(224,224),n_labels=5))
  test_data = tf.data.Dataset.from_tensor_slices(data(rgb,mask,depth,liste=test_list,dims=(224,224),n_labels=5))




In [ ]:
def compute_mean_std(image_paths):
    mean = None
    M2 = None
    count = 0

    for path in image_paths:
        img = read_exr(path)
        img = img.astype(np.float32) / 255.0 

        if mean is None:
            mean = np.mean(img, axis=(0, 1))
        else:
            mean += np.mean(img, axis=(0, 1))

        count += 1

    mean /= count

    for path in image_paths:
        img = read_exr(path).astype(np.float32) / 255.0
        variance = np.mean((img - mean) ** 2, axis=(0, 1))

        if M2 is None:
            M2 = variance
        else:
            M2 += variance

    std = np.sqrt(M2 / count)

    return mean, std

mean, std = compute_mean_std(rgb)
print("Mean:", mean)
print("Std:", std)

Mean: [0.0017513  0.00183881 0.00178873]
Std: [0.25150472 0.2346499  0.15039594]


In [ ]:
def dice_coef(y_true, y_pred):
    y_true = K.reshape(y_true, K.shape(y_pred))
    epsilon=1e-6
    axes = tuple(range(1, len(y_pred.shape)-1))
    numerator = 2. * K.sum(y_pred * y_true, axes)
    denominator = K.sum(K.square(y_pred) + K.square(y_true), axes)

    return K.mean((numerator + epsilon) / (denominator + epsilon))

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

In [ ]:
with strategy.scope():
  model.compile(loss=dice_coef_loss, optimizer=keras.optimizers.Adam(learning_rate=0.0001), metrics=["accuracy", dice_coef])

In [ ]:
model.load_weights('/content/drive/MyDrive/Colab Notebooks/segnet/model_weights_unet.weights.h5')

In [ ]:
log_dir = "/content/profiler_"#"/content/drive/MyDrive/Colab Notebooks/tensor_board"
profiler = tf.keras.callbacks.TensorBoard(log_dir = log_dir, profile_batch = (10,100) )

In [ ]:
def serialize_example(image, mask):
    feature = {
        "image": tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.serialize_tensor(image).numpy()])),
        "mask": tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.serialize_tensor(mask).numpy()]))
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

tfrecord_path = "/content/drive/MyDrive/val_dataset.tfrecord2"

with tf.io.TFRecordWriter(tfrecord_path) as writer:
    for image, mask in val_data:
        example = serialize_example(image, mask)
        writer.write(example)


In [ ]:
def parse_tfrecord_fn(example):
    feature_description = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "mask": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, feature_description)
    image = tf.io.parse_tensor(example["image"], out_type=tf.float32)
    mask = tf.io.parse_tensor(example["mask"], out_type=tf.float32)
    return image, mask

try_dataset = tf.data.TFRecordDataset(tfrecord_path)
try_dataset = try_dataset.map(parse_tfrecord_fn).batch(16*8)  # Указываем размер батча


In [ ]:
train_data = train_data.batch(16*8,drop_remainder=True).prefetch(buffer_size = tf.data.AUTOTUNE)
val_data = val_data.batch(16*8,drop_remainder=True).prefetch(buffer_size = tf.data.AUTOTUNE)
test_data = test_data.batch(1).prefetch(buffer_size = tf.data.AUTOTUNE)

In [ ]:
checkpoint_cb = ModelCheckpoint(
    filepath= "/content/weights_/model_weights_unet.weights.h5",#"/content/drive/MyDrive/Colab Notebooks/segnet/model_weights_unet.weights.h5", #"/content/drive/MyDrive/Colab Notebooks/segnet/model_weights.weights.h5",  #model_weights2.weights.h5
    save_best_only=True,
    save_weights_only=True,
    monitor="val_loss",
    mode="min"
)

history = model.fit(
    train_data,
    epochs=50,
    validation_data=val_data,
    callbacks = [checkpoint_cb,profiler]
)


In [ ]:
plt.plot(history.history['accuracy'],markersize=20)
plt.plot(history.history['val_accuracy'], markersize=20)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
# plotting of training and validation loss curves
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
def convert_exr_to_rgb(exr_path):
    exr_file = OpenEXR.InputFile(exr_path)

    header = exr_file.header()
    data_window = header['dataWindow']
    width = data_window.max.x - data_window.min.x + 1
    height = data_window.max.y - data_window.min.y + 1

    channels = ['R', 'G', 'B']
    rgb_data = [np.frombuffer(exr_file.channel(channel, Imath.PixelType(Imath.PixelType.FLOAT)), dtype=np.float32)
                for channel in channels]

    rgb_array = [np.clip(channel.reshape(height, width) * 255, 0, 255).astype(np.uint8) for channel in rgb_data]

    rgb_image = np.dstack(rgb_array)
    return rgb_image

In [ ]:
h=0
for i in [0,1]:#test_list:
    if h<1:

        img_path = "/content/dataset of synthetic rgb-d plants/rgb_map/rgb_map_part_1/rgb_map/Image5661.exr"#rgb[i]
        original_img = convert_exr_to_rgb(img_path)
        plt.figure(figsize=(15,15))
        plt.subplot(1,2,1)
        plt.imshow(original_img)
        array_img=img_to_array(original_img)/255

        array_img2 = np.reshape(array_img, (1,224,224,3))
        y_pred=model.predict(array_img2)
        y_pred = np.reshape(y_pred[0], (224, 224, 5))
        c=class_pixels(y_pred)
        o=colorize(c)
        plt.subplot(1,2,2)
        plt.imshow(o)
        plt.show()
        h+=1
        print("Original Image Shape:", array_img.shape)
        print("Predicted Shape:", y_pred.shape)
        print("Class Pixels Shape:", c.shape)
        print("Colorized Output Shape:", o.shape)

    else:
        break